In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras


### Activation Functions
        sigmoid, tanh, relu, leaky_relu
Use Sigmoid in output layer. All other places try to use Tanh

For hidden layers if you are not sure which activation functions to use, just use ReLU as your default choice

Con of Sigmoid, Tanh, ReLU => Vanishing Gradients

Leaky ReLU => Does not have Vanishing Gradients

<!-- <img src='./images/activation_functions.png' style="height: 550px; width: 1200px;"></img> -->


In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanhh(x):
    return (np.exp(2*x) - 1) / (np.exp(2*x) + 1)

def relu(x):
    return max(0,x)

def leaky_relu(x, a=0.1):
    return max(a*x, x)

### Loss [L(f(x), y)] & Cost [J(w, b)] Functions
    sparse_categorical_crossentropy, binary_crossentropy[logistic_loss], categorical_crossentropy, mean_absolute_error, mean_squared_error
    
Loss function is to capture the difference between the actual and predicted values for a single record whereas Cost functions aggregate the difference for the entire training dataset.

Epoch -> Going through all the training samples once.



<!-- <img src='./images/Loss&CostFunctions.png' style="height:100px; width: 330px;"></img> -->


In [6]:
def mean_absolute_error(y_true, y_predicted):
    return np.mean(np.abs(y_predicted - y_true))

def mean_squared_error(y_true, y_predicted):
    return np.mean(np.square(y_true - y_predicted))

def binary_crossentropy(y_true, y_predicted):
    ep = 1e-15
    y_predicted_new = [max(i,ep) for i in y_predicted]
    y_predicted_new = [min(i,1-ep) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

### Gradient Descent

In [7]:
df = pd.read_csv('./data/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [8]:
scalar = MinMaxScaler()
scalar.fit(df[['age']]) # scaling 'age' feature
df['age'] = scalar.transform(df[['age']])
df.head()

,age,affordibility,bought_insurance
0,0.090909,1,0
1,0.159091,0,0
2,0.659091,1,1
3,0.772727,0,0
4,0.636364,1,1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['bought_insurance'], axis='columns'), df['bought_insurance'], test_size=0.2, random_state=10) 
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(22, 2) (6, 2) (22,) (6,)


In [10]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']  
)

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 403ms/step - loss: 0.7085 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 7ms/step - loss: 0.7081 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 7ms/step - loss: 0.7076 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 7ms/step - loss: 0.7072 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7067 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7063 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7058 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.7054 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7050 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 156ms/step - loss: 0.7045 - accuracy: 0.5000
Epoch 11/100
1/1 [=======

In [11]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 128ms/step - loss: 0.5748 - accuracy: 0.5000


[0.5747960805892944, 0.5]

In [12]:
model.predict(X_test)

1/1 [==============================] - 0s 64ms/step


array([[0.6832469 ],
       [0.51686794],
       [0.83058196],
       [0.8028394 ],
       [0.53229773],
       [0.6912613 ]], dtype=float32)

In [13]:
y_test

7     1
21    0
5     1
2     1
13    0
19    0
Name: bought_insurance, dtype: int64

In [14]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.90748274],
        [0.90352213]], dtype=float32),
 array([-0.09749975], dtype=float32))

In [15]:
df = pd.read_csv('./data/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [16]:
def min_max_scaled(df):
    df_arr = df.values.astype(float)
    df_normalized = (df_arr - df_arr.min(axis=0)) / (df_arr.max(axis=0) - df_arr.min(axis=0))
    df = pd.DataFrame(df_normalized, columns=df.columns)
    return df

In [17]:
df = min_max_scaled(df)
df.head()

,age,affordibility,bought_insurance
0,0.090909,1.0,0.0
1,0.159091,0.0,0.0
2,0.659091,1.0,1.0
3,0.772727,0.0,0.0
4,0.636364,1.0,1.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['bought_insurance'], axis='columns'), df['bought_insurance'], test_size=0.2, random_state=10) 
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(22, 2) (6, 2) (22,) (6,)


In [32]:
def gradient_descent(X, y, epoch=100, loss_threshold=0.2, learning_rate=0.05):
    n = X.shape[1] 
        
    w = np.ones(n)
    b = 0

    for i in range(epoch):
        weighted_sum = w@X.T + b
        y_predicted = sigmoid(weighted_sum)
        loss = binary_crossentropy(y, y_predicted)
        
        dw = (1/n) * X.T @ (y_predicted - y)
        db = np.mean(y_predicted - y)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
    
        if loss < loss_threshold:
            print(f"Converged after {i} epochs")
            break
            
    return w,b

In [33]:
w, b = gradient_descent(X_train, y_train)
print(w, b)

age              1.740879
affordibility    0.151551
dtype: float64 -0.5761554471202169


In [34]:
def accuracy(X_test, y_test, w, b):
    weighted_sum =  w@X_test.T + b
    
    y_predicted = sigmoid(weighted_sum)
    
    y_predicted_class = np.round(y_predicted)
    accuracy = np.mean(y_predicted_class == y_test)
    
    return accuracy

In [35]:
print("Accuracy:", 100*round(accuracy(X_test, y_test, w, b),5), "%")

Accuracy: 100.0 %


### Neural Network

In [36]:
class myNeuralNetwork:
    def fit(self, X, y):
        self.w, self.bias = self.gradient_descent(X,y)
        print(f"Final weights and bias: w:{self.w.values}, bias: {self.bias}")
        
    def predict(self, X):
        weighted_sum = self.w@X.T + self.bias
        return sigmoid(weighted_sum)
    
    def gradient_descent(self, X, y, epoch=100, loss_threshold=0.2, learning_rate=0.05):
        n = X.shape[1] 

        self.w = np.ones(n)
        self.b = 0

        for i in range(epoch):
            weighted_sum = self.w@X.T + self.b
            y_predicted = sigmoid(weighted_sum)
            loss = binary_crossentropy(y, y_predicted)

            dw = (1/n) * X.T @ (y_predicted - y)
            db = np.mean(y_predicted - y)

            self.w -= learning_rate * dw
            self.b -= learning_rate * db

            if i%50==0:
                print (f'Epoch:{i}, w:{self.w.values}, bias:{self.b}, loss:{loss}\n')
            
            if loss<=loss_threshold:
                print (f'Epoch:{i}, w:{self.w.values}, bias:{self.b}, loss:{loss}\n')
                break

        return self.w,self.b
    
    def accuracy(self, X_test, y_test):
        y_predicted = self.predict(X_test)
        y_predicted_class = np.round(y_predicted)
        accuracy = np.mean(y_predicted_class == y_test)
        return accuracy


In [37]:
myModel = myNeuralNetwork()
myModel.fit(X_train, y_train)

Epoch:0, w:[0.97335279 0.92189305], bias:-0.012825144365159014, loss:0.7085138021259431

Epoch:50, w:[1.30740092 0.161214  ], bias:-0.33847453079770234, loss:0.5876411969479016

Final weights and bias: w:[1.7408789  0.15155116], bias: -0.5761554471202169


In [38]:
print("Accuracy:", 100*round(myModel.accuracy(X_test, y_test),5), "%")

Accuracy: 100.0 %
